<h1>Importing Libraries

In [1]:
#https://www.youtube.com/watch?v=06TE_U21FK4&t=1199s, using this video to learn this
import cv2 as cv
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>1. Accessing webcam and displaying it

In [2]:
# Open the webcam (0 indicates the default webcam)
cap = cv.VideoCapture(0)

# Use the MediaPipe Pose function with specified confidence thresholds for detection and tracking
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():  # Continue reading frames as long as the webcam is open
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:  # If frame reading fails, break the loop
            break

        # Convert the frame from BGR (OpenCV format) to RGB (MediaPipe format)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False  # Set the image to read-only to improve performance

        # Process the image to detect pose landmarks
        results = pose.process(image)

        # Convert the processed image back to BGR for OpenCV
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        image.flags.writeable = True  # Set the image to writable

        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Draw the pose landmarks on the image if landmarks are detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(76, 76,241 ), thickness= 2, circle_radius=2), #for landmarks
                mp_drawing.DrawingSpec(color=(139, 255, 144), thickness= 2, circle_radius=2) # for connection
                )#Here the color is in bgr format

        # Display the image with landmarks in a window named "Webcam Feed"
        cv.imshow("Webcam Feed", image)

        # Break the loop if the 'q' key is pressed
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

# Release the webcam and close the OpenCV window
cap.release()

cv.destroyAllWindows()

c:\Users\Yokai\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<h1>2. Determining Joints</h1>

In [3]:
len(landmarks)

33

In [ ]:
# Here we are printing which point gives which landmark
for i in mp_pose.PoseLandmark:
    print(i.name)

In [5]:
mp_pose.PoseLandmark.LEFT_SHOULDER

<PoseLandmark.LEFT_SHOULDER: 11>

<h3>Grabbing landmarks from our video feed

In [6]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.674743474
y: 0.952904463
z: -0.194127455
visibility: 0.998709142

In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.759268522
y: 1.24112356
z: -0.194696292
visibility: 0.0755195841

In [8]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.759255469
y: 1.46481061
z: -0.451325506
visibility: 0.0177333746

<h1>3. Calculating Angles

In [18]:
def calculateAngle(a, b, c):
    # Convert the points to NumPy arrays for vectorized operations
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Calculate the angle in radians using the arctan2 function
    # The arctan2 function returns the arctangent of the quotient of its arguments, taking into account the sign to determine the correct quadrant
    radians = np.arctan2(b[1] - a[1], b[0] - a[0]) - np.arctan2(b[1] - c[1], b[0] - c[0])
    
    # Convert the angle from radians to degrees
    angle = np.abs(radians * 180.0 / np.pi)

    # Adjust the angle to be within the range of 0 to 180 degrees
    if angle > 180.0:
        angle = 360 - angle
    
    # Return the calculated angle
    return angle

In [11]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
shoulder

[0.6747434735298157, 0.952904462814331]

In [13]:
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
elbow

[0.7592685222625732, 1.2411235570907593]

In [14]:
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
wrist

[0.7592554688453674, 1.4648106098175049]

In [20]:
calculateAngle(shoulder, elbow, wrist)

np.float64(163.65200286227727)

In [23]:
# Open the webcam (0 indicates the default webcam)
cap = cv.VideoCapture(0)

# Use the MediaPipe Pose function with specified confidence thresholds for detection and tracking
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():  # Continue reading frames as long as the webcam is open
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:  # If frame reading fails, break the loop
            break

        # Convert the frame from BGR (OpenCV format) to RGB (MediaPipe format)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False  # Set the image to read-only to improve performance

        # Process the image to detect pose landmarks
        results = pose.process(image)

        # Convert the processed image back to BGR for OpenCV
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        image.flags.writeable = True  # Set the image to writable

        try:
            landmarks = results.pose_landmarks.landmark

            # taking landmarks
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # calculating angle
            angle = calculateAngle(shoulder, elbow, wrist)

            # Visualize
            cv.putText(img=image,
                       text=str(angle),
                       org=tuple(np.multiply(elbow, [640, 480]).astype(int)),
                       fontFace=cv.FONT_HERSHEY_SIMPLEX,
                       fontScale=0.5, color=(255, 255, 255),
                       thickness=2,
                       lineType=cv.LINE_AA
                       )
        except:
            pass

        # Draw the pose landmarks on the image if landmarks are detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(
                    # for landmarks
                    color=(76, 76, 241), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(
                    # for connection
                    color=(139, 255, 144), thickness=2, circle_radius=2)
            )  # Here the color is in bgr format

        # Display the image with landmarks in a window named "Webcam Feed"
        cv.imshow("Webcam Feed", image)

        # Break the loop if the 'q' key is pressed
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

# Release the webcam and close the OpenCV window
cap.release()

cv.destroyAllWindows()